<a href="https://colab.research.google.com/github/kethlyncampos/webscraper/blob/main/web_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt update
!apt install chromium-chromedriver
!pip install selenium
!pip install requests
!pip install iso8601

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

     |████████████████████████████████| 127 kB 28.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.7
    Uninstalling urllib3-1.26.7:
      Successfully uninstalled urllib3-1.26.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.0 requires urllib3[secure]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [1]:
import requests
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
import pytz
from datetime import datetime
import iso8601
from dateutil import tz
from google.colab import drive
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.request import Request, urlopen

In [2]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
yahoo = pd.read_csv('/content/gdrive/MyDrive/news_yahoo.csv')
infomoney = pd.read_csv('/content/gdrive/MyDrive/news_infomoney.csv', lineterminator='\n')
investing = pd.read_csv('/content/gdrive/MyDrive/news_investing.csv')

In [5]:
comp_yahoo = []
comp_infomoney = []
comp_investing = []

for c in yahoo['URL']:
  comp_yahoo.append(c)

for c in infomoney['URL']:
  comp_infomoney.append(c)

for c in investing['URL']:
  comp_investing.append(c)

In [8]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
options.add_argument("start-maximized")
options.add_argument('disable-infobars')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36")
browser = webdriver.Chrome(options=options)

In [9]:
def req_yahoo(url):
  browser.get(url)
  scrollar(browser)
  links = browser.find_elements(By.CSS_SELECTOR, 'a.js-content-viewer.rapidnofollow')
  noticias = []

  for link in links:
    if (('https://br.yahoo.com/news/' not in link.get_attribute('href')) and (link.get_attribute('href') not in comp_yahoo)):
      browser.execute_script("arguments[0].click();", link)
      scrollar(browser)
      WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
      temp = BeautifulSoup(browser.page_source, 'html.parser')
      title = temp.find('h1', attrs={'data-test-locator':'headline'}).text
      author = temp.find('span', attrs={'class':'caas-author-byline-collapse'}).text
      collect_date = convert_timezone(datetime.now())
      publication_date = convert_timezone(iso8601.parse_date(temp.find('time')['datetime']))
      description = temp.find('div', attrs={'class':'caas-body'})
      try:
        description.figure.decompose()
        description = description.text
      except:
        description = description.text
      noticias.append(['Yahoo Finanças',title, description, author, publication_date, collect_date, browser.current_url])
      browser.back()
      sleep(3)
  browser.close()
  return noticias

In [21]:
def req_infomoney(url):
  browser = webdriver.Chrome(options=options)
  browser.get(url)
  browser.maximize_window()
  infinite = browser.find_element(By.XPATH, '//*[@id="infinite-handle"]/span/button')
  browser.execute_script("arguments[0].click();", infinite)
  count = 10
  while(count > 0):
    try:
      scrollar(browser)
      WebDriverWait(browser, 40).until(EC.presence_of_element_located((By.XPATH, '//*[@id="infinite-handle"]/span/button')))
      browser.execute_script("arguments[0].click();", infinite)
      sleep(4)
      count-=1
    except:
      break
  page = BeautifulSoup(browser.page_source, 'html.parser')
  browser.close()
  links = page.find_all(class_="hl-title")
  lista_url = []
  for j in links:
    aux = j.find('a')
    if(aux):
      if "stock-pickers" not in aux['href']:
        lista_url.append(aux['href'])
  lista_url = set(lista_url)
  noticias = []
  subtitle = ""
  for k in lista_url:
    if k not in comp_infomoney:
      response = requests.get(k)
      temp = BeautifulSoup(response.content, 'html.parser')
      title = temp.find('h1', attrs={'class': 'page-title-1'})
      if title: # se não houver título, a página é do tipo cotações e possui apenas as informações sobre a empresa
        title = title.text
        subtitle = trata_string(temp.find('p', attrs={'class': 'article-lead'}).text)
        author = temp.find('span', attrs={'class':'author-name'})
        if author:
          author = trata_string(author.find('a').text) # se não houver autor, a página é do tipo guias
          collect_date = convert_timezone(datetime.now())
          publication_date = iso8601.parse_date(temp.find('time', attrs={'class':'entry-date'})['datetime']).strftime('%Y-%m-%d %H:%M:%S')
          description = temp.find('div', attrs={'class':'article-content'})
          try:
            description.figure.decompose()
            description = description.text
          except:
            description = description.text
          description = trata_string(description)
          noticias.append(['Infomoney',title, subtitle, description, author, publication_date, collect_date, k])
  return noticias

In [11]:
def req_investing(url):
  browser2 = webdriver.Chrome(options=options)
  browser2.get(url)
  scrollar(browser2)
  noticias = []
  while(True):
    try:
      temp = BeautifulSoup(browser2.page_source, 'html.parser')
      links = temp.find_all('article', attrs={'class':'js-article-item'})
      aux = []
      for i in links:
        path = i.find('div', attrs={'class':'textDiv'})
        path = path.find('a')['href']
        if "https://br.investing.com" not in path:
          path = "https://br.investing.com"+ path
        if ("https://br.investing.com/news/stock-market-news" in path) and ("#comments" not in path) and (path not in comp_investing):
          aux.append(path)
      for link in aux:
        req = Request(link , headers={'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        print(link)
        temp = BeautifulSoup(webpage, "html.parser")
        title = temp.find('h1', attrs={'class':'articleHeader'}).text
        time = temp.find('div', attrs={'class':'contentSectionDetails'})
        time = time.find('span').text
        try:
          time = re.search('\(([^)]+)', time).group(1)
        except:
          pass
        publication_date = datetime.strptime(time, '%d.%m.%Y %H:%M')
        if datetime.strftime(publication_date, '%Y') == "2018":
          break
        publication_date = datetime.strftime(publication_date, '%Y-%m-%d %H:%M:%S')
        collect_date = convert_timezone(datetime.now())
        description = temp.find('div', attrs={'class':'articlePage'})
        try:
          description.find('div',class_='imgCarousel').decompose()
          description = description.text
        except:
          description = description.text
        description = trata_string(description)
        noticias.append(['Investing.com',title, description, publication_date, collect_date, link])
        comp_investing.append(link)

      proximo = browser2.find_element(By.XPATH, '//*[@id="paginationWrap"]/div[3]/a')
      browser2.execute_script("arguments[0].click();", proximo)
      scrollar(browser2)
    except:
      break

  browser2.close()
  return noticias

In [12]:
def scrollar(driver):
  lastHeight = driver.execute_script("return document.body.scrollHeight")
  while True:
      driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
      sleep(5)
      newHeight = driver.execute_script("return document.body.scrollHeight")
      if newHeight == lastHeight:
          break
      lastHeight = newHeight
  return

In [13]:
def convert_timezone(date_time):
  sp = tz.gettz('America/Sao_Paulo')
  new_date = date_time.replace(tzinfo=pytz.utc)
  new_date = new_date.astimezone(sp)
  new_date = new_date.strftime('%Y-%m-%d %H:%M:%S')
  return new_date

In [14]:
def trata_string(data):
  data = re.sub(r'[\t\n ]+', ' ', data)
  return data

In [15]:
array_noticias = req_yahoo('https://br.yahoo.com/topics/bolsa-de-valores/')

In [16]:
dataset_temp = pd.DataFrame(array_noticias,columns=['Fonte', 'Título', 'Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [17]:
if len(dataset_temp) > 0:
  dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)
  dataset_temp.reset_index(drop = True, inplace = True)
  yahoo = pd.concat([dataset_temp,yahoo],ignore_index=True)
  yahoo.drop_duplicates(subset=['Descrição'], inplace = True)
  yahoo.drop(yahoo.filter(regex="Unname"),axis=1, inplace=True)
  yahoo = yahoo.reset_index(drop=True)
  yahoo.to_csv('/content/gdrive/MyDrive/news_yahoo.csv', index=False)

In [22]:
noticias_info = req_infomoney("https://www.infomoney.com.br/mercados/")

In [23]:
dataset_temp = pd.DataFrame(noticias_info,columns=['Fonte', 'Título', 'Subtítulo','Descrição', 'Autor', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [24]:
if len(dataset_temp) > 0:
  dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)
  dataset_temp.reset_index(drop = True, inplace = True)
  infomoney = pd.concat([dataset_temp, infomoney])
  infomoney.drop_duplicates(subset=['Descrição'], inplace = True)
  infomoney.drop(infomoney.filter(regex="Unname"),axis=1, inplace=True)
  infomoney = infomoney.reset_index(drop=True)
  infomoney.to_csv('/content/gdrive/MyDrive/news_infomoney.csv', index=False)

In [ ]:
page2 = req_investing('https://br.investing.com/news/stock-market-news/76')

https://br.investing.com/news/stock-market-news/tripulacao-da-nasaspacex-volta-de-missao-recorde-na-estacao-espacial-internacional-939553
https://br.investing.com/news/stock-market-news/acoes-da-tesla-caem-apos-usuarios-do-twitter-votam-para-musk-vender-fatia-939150
https://br.investing.com/news/stock-market-news/petz-tem-lucro-liquido-de-r266-mi-no-3o-tri-preve-abertura-de-50-lojas-em-2022-939464
https://br.investing.com/news/stock-market-news/banco-do-brasil-quer-aumentar-carteira-de-credito-com-produtos-mais-arriscados-939465
https://br.investing.com/news/stock-market-news/pec-dos-precatorios-da-o-tom-e-ibovespa-abre-em-alta-939533
https://br.investing.com/news/stock-market-news/taxas-do-tesouro-direto-recuam-com-expectativa-por-pec-dos-precatorios-939531
https://br.investing.com/news/stock-market-news/apesar-de-ny-ibovespa-sobe-acima-dos-106-mil-pontos-de-olho-em-pec-939529
https://br.investing.com/news/stock-market-news/general-electric-anuncia-que-dividira-empresa-em-tres-939512


In [18]:
dataset_temp = pd.DataFrame(page2,columns=['Fonte', 'Título', 'Descrição', 'Data/Hora de Publicação', 'Data/Hora de Coleta', 'URL'])

In [20]:
if len(dataset_temp) > 0:
  dataset_temp.drop_duplicates(subset=['Descrição'], inplace = True)
  dataset_temp.reset_index(drop = True, inplace = True)
  investing = pd.concat([dataset_temp, investing])
  investing = dataset_temp
  investing.drop_duplicates(subset=['Descrição'], inplace = True)
  investing.drop(investing.filter(regex="Unname"),axis=1, inplace=True)
  investing = investing.reset_index(drop=True)
  investing.to_csv('/content/gdrive/MyDrive/news_investing.csv', index=False)